In [11]:
%profile "cloudbend"

%glue_version "3.0"
%iam_role "arn:aws:iam::898546127587:role/GlueSessions"
%idle_timeout 60

%worker_type "G.1X"
%number_of_workers 5

%extra_jars "s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-client-2.17.257.jar"

Previous profile: None
Setting new profile to: cloudbend
Setting Glue version to: 3.0
Current iam_role is None
iam_role has been set to arn:aws:iam::898546127587:role/GlueSessions.
Current idle_timeout is 2880 minutes.
idle_timeout has been set to 60 minutes.
Previous worker type: G.1X
Setting new worker type to: G.1X
Previous number of workers: 5
Setting new number of workers to: 5


Additional python modules to be included:
tqdm
Extra jars to be included:
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-client-2.17.257.jar
s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-client-2.17.257.jar


In [13]:
%%configure
{
    "conf": "spark.jars.packages=org.apache.iceberg:iceberg-spark-runtime-3.0_2.12-1.0.0,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257 --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
}

The following configurations have been updated: {'conf': 'spark.jars.packages=org.apache.iceberg:iceberg-spark-runtime-3.0_2.12-1.0.0,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257 --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions'}


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.getActiveSession()

Authenticating with profile=cloudbend
glue_role_arn defined by user: arn:aws:iam::898546127587:role/GlueSessions
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: 202f883b-5df5-4423-9900-4a2933a4ae8a
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
--additional-python-modules tqdm
--extra-jars s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/iceberg/iceberg-spark-runtime-3.0_2.12-1.0.0.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/bundle-2.17.257.jar,s3://lambdaicebergdemo-icebergbucket89dd3fa6-fyq2q519smob/jars/aws/url-connection-client-2.17.257.jar
--conf spark.jars.packages=org.apache.iceberg:iceberg-spark-runtime-3.0_2.12-1.0.0,software.amazon.awssdk:bundle:2.17.257,software.amazon.awssdk:url-connection-client:2.17.257 --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions
Waiting for se

In [2]:
catalog = "iceberg"
bucket = "pyicebergbenchmark-icebergbucket89dd3fa6-1qhkzajoxgpzo"
prefix = "iceberg"

iceberg_conf = [
    (f"spark.sql.catalog.{catalog}", "org.apache.iceberg.spark.SparkCatalog"),
    (f"spark.sql.catalog.{catalog}.warehouse", f"s3://{bucket}/{prefix}/{catalog}/"),
    (f"spark.sql.catalog.{catalog}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog"),
    (f"spark.sql.catalog.{catalog}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO"),
]

for k, v in iceberg_conf:
    spark.conf.set(k, v)

In [ ]:
df = (
    spark.read
        .option("mergeSchema", "true")
        .parquet(f"s3://{bucket}/raw/*/")
)

df.printSchema()

In [57]:
df.createOrReplaceTempView("nyc_taxi")

In [49]:
%%sql

SELECT COUNT(DISTINCT dispatching_base_num)
FROM nyc_taxi;

+------------------------------------+
|count(DISTINCT dispatching_base_num)|
+------------------------------------+
|                                1333|
+------------------------------------+


In [58]:
%%sql

CREATE DATABASE IF NOT EXISTS iceberg.benchmark;

++
||
++
++


In [59]:
%%sql

CREATE TABLE iceberg.benchmark.nyc_taxi
USING iceberg
PARTITIONED BY (months(pickup_datetime))
TBLPROPERTIES (
    'write.parquet.compression-codec' = 'snappy',
    'write.metadata.metrics.default' = 'full'
)
AS
SELECT *
FROM nyc_taxi
ORDER BY pickup_datetime;

Py4JJavaError: An error occurred while calling o69.sql.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 195 in stage 60.0 failed 4 times, most recent failure: Lost task 195.3 in stage 60.0 (TID 4458) (172.34.78.30 executor 3): java.lang.UnsupportedOperationException: org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainBinaryDictionary
	at org.apache.parquet.column.Dictionary.decodeToLong(Dictionary.java:49)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetDictionary.decodeToLong(ParquetDictionary.java:36)
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.getLong(OnHeapColumnVector.java:364)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:755)
	at scal

In [14]:
%stop_session

Stopping session: 2858e40c-8d41-43dd-9c24-61afddfbeefe
Stopped session.
